In [1]:
%load_ext tutormagic

# Special Method Names

## Special Method Names in Python

Certain names in Python are special because they have built-in behavior. These names always start and end with 2 underscores `__`. 

| Name | Method invoked... |
| --- | --- |
| `__init__` | ...automatically when an object is constructed |
| `__repr__` | ...to display an object as a Python expression |
| `__add__` | ...to add one object to another |
| `__bool__` | ...to convert an object to `True` or `False`|
| `__float__` | ...to convert an object to a float (real number) |

In [2]:
zero, one, two = 0, 1, 2
one + two

3

In [3]:
bool(zero), bool(one)

(False, True)

It's possible to rewrite the entire sequence in a different way.

<img src = 'same.jpg' width = 800/>

Above is just another example of using interface to allow user-defined objects to interact with built-in systems in Python.

#### What happens when we have 2 instances of user-defined classes added together?

Adding instances of user-defined classes invokes either the `__add__` or `__radd__` method. 

Thus, the 2 expressions below are equivalent,

In [ ]:
>>> Ratio(1, 3) + Ratio(1, 6)
Ratio(1, 2)

In [ ]:
>>> Ratio(1, 3).__add__(Ratio(1, 6))
Ratio(1, 2)

Obviously, the first expression is the more commonly used. The purpose of writing the second one is to study and use the method definition syntax to override the `+` symbol. 

We can also use `__radd__`. The difference is that the arguments are switched. However, this is trivial since addition is cumulative anyway.

In [ ]:
>>> Ratio(1, 6).__radd__(Ratio(1, 3))
Ratio(1, 2)

## Demo - Add 2 instances of user-defined classes

Recall our `Ratio` class.

In [1]:
class Ratio:
    def __init__(self, n, d):
        self.numer = n # numerator instance attribute
        self.denom = d # denominator instance attribute
        
    def __repr__(self):
        return 'Ratio({0}, {1})'.format(self.numer, self.denom)
    
    def __str__(self):
        return '{0}/{1}'.format(self.numer, self.denom)

We will add the `_add__` method, which computes the numerator and the denominator of the result. Recall that,

$$\frac{x1}{y1} + \frac{x2}{y2} = \frac{x1 \times y2  + x2 \times y1}{y1 \times y2}$$

And we can reduce the fraction by dividing both the numerator and denominator with the greatest common divisor.

In [4]:
def gcd(n, d):
    while n != d:
        n, d = min(n, d), abs(n - d)
    return n

In [10]:
class Ratio:
    def __init__(self, n, d):
        self.numer = n # numerator instance attribute
        self.denom = d # denominator instance attribute
        
    def __repr__(self):
        return 'Ratio({0}, {1})'.format(self.numer, self.denom)
    
    def __str__(self):
        return '{0}/{1}'.format(self.numer, self.denom)
    
    def __add__(self, other):
        n = self.numer * other.denom + self.denom * other.numer
        d = self.denom * other.denom
        g = gcd(n, d)
        return Ratio(n//g, d//g)

In [11]:
Ratio(1, 3) + Ratio(1, 6)

Ratio(1, 2)

The addition works! However, what if we add a ratio with an integer?

In [7]:
Ratio(1, 3) + 1

AttributeError: 'int' object has no attribute 'denom'

Then we'll obtain an error saying that an `int` object does not have the attribute `denom`. 

Now recall that if we add a fraction with an integer, for example,

$$ \frac{1}{3} + 2 = \frac{1}{3} + \frac{2 \times 3}{3} $$

Or in other words,

$$ \frac{n}{d} + x = \frac{n}{d} + \frac{x \times d}{d} $$

We can modify the `__add__` method so that if the `other` argument is an integer, it would be converted to a fraction.

In [21]:
class Ratio:
    def __init__(self, n, d):
        self.numer = n # numerator instance attribute
        self.denom = d # denominator instance attribute
        
    def __repr__(self):
        return 'Ratio({0}, {1})'.format(self.numer, self.denom)
    
    def __str__(self):
        return '{0}/{1}'.format(self.numer, self.denom)
    
    def __add__(self, other):
        if isinstance(other, int): # if 'other' is an int
            n = self.numer + other * self.denom
            d = self.denom
        else: # if 'other' is a fcraction
            n = self.numer * other.denom + self.denom * other.numer
            d= self.denom * other.denom
            
        g = gcd(n,d)
        return Ratio(n//g, d//g)

In [22]:
Ratio(1, 3) + 1

Ratio(4, 3)

Addition with integer works! However, will it work if we start with an integer?

In [23]:
1 + Ratio(1, 3)

TypeError: unsupported operand type(s) for +: 'int' and 'Ratio'

It doesn't work!

We can specify in the class that the right hand addition (`__rad__`) is the same as normal addition (`__add__`),

In [24]:
class Ratio:
    def __init__(self, n, d):
        self.numer = n # numerator instance attribute
        self.denom = d # denominator instance attribute
        
    def __repr__(self):
        return 'Ratio({0}, {1})'.format(self.numer, self.denom)
    
    def __str__(self):
        return '{0}/{1}'.format(self.numer, self.denom)
    
    def __add__(self, other):
        if isinstance(other, int): # if 'other' is an int
            n = self.numer + other * self.denom
            d = self.denom
        else: # if 'other' is a fcraction
            n = self.numer * other.denom + self.denom * other.numer
            d= self.denom * other.denom
            
        g = gcd(n,d)
        return Ratio(n//g, d//g)
    
    __radd__ = __add__ # Specify that the right hand addition is the same as normal addition

Now if we try to start with integer again,

In [25]:
1 + Ratio(1, 3)

Ratio(4, 3)

Now what if we involve a float?

In [26]:
1.02 + Ratio(1, 3)

AttributeError: 'float' object has no attribute 'denom'

In the case of float, we can have the final result of the calculation in float instead of `Ratio`. We would add the condition,

In [ ]:
elif instance(other, float):
    return float(self) + other

Note that the condition above involves calling `float` on `self`, which means to convert `self` to a float. We would need to define this `float` method, which easily returns the numerator divided by the denominator.

In [ ]:
def __float__(self):
    return self.numer / self.denom

In [29]:
class Ratio:
    def __init__(self, n, d):
        self.numer = n # numerator instance attribute
        self.denom = d # denominator instance attribute
        
    def __repr__(self):
        return 'Ratio({0}, {1})'.format(self.numer, self.denom)
    
    def __str__(self):
        return '{0}/{1}'.format(self.numer, self.denom)
    
    def __add__(self, other):
        if isinstance(other, int): # if 'other' is an int
            n = self.numer + other * self.denom
            d = self.denom
        elif isinstance(other, float): # if 'other' is a float
            return float(self) + other
        else: # if 'other' is a fcraction
            n = self.numer * other.denom + self.denom * other.numer
            d= self.denom * other.denom
            
        g = gcd(n,d)
        return Ratio(n//g, d//g)
    
    def __float__(self):
        return self.numer / self.denom
    
    __radd__ = __add__ # Specify that the right hand addition is the same as normal addition

In [28]:
0.2 + Ratio(1, 3)

0.5333333333333333

Above, we applied 2 important ideas:

#### 1. Type Dispatching

In [ ]:
if isinstance(other, int): # if 'other' is an int
    n = self.numer + other * self.denom
    d = self.denom
elif isinstance(other, float): # if 'other' is a float
    return float(self) + other
else: # if 'other' is a fcraction
    n = self.numer * other.denom + self.denom * other.numer
    d= self.denom * other.denom

In type dispatching, we inspect the type of the argument to decide what to do. 

#### 2. Type Cohersion

In [ ]:
return float(self) + other

Here we take an object of certain type and convert it to another type.